In [1]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
from sqlalchemy import *
import pudl
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Connecting to database

In [2]:
#Setting up our workspace
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_settings

{'pudl_in': '/Users/andreaquevedo/Desktop/PUDL Data',
 'data_dir': '/Users/andreaquevedo/Desktop/PUDL Data/data',
 'settings_dir': '/Users/andreaquevedo/Desktop/PUDL Data/settings',
 'pudl_out': '/Users/andreaquevedo/Desktop/PUDL Data',
 'sqlite_dir': '/Users/andreaquevedo/Desktop/PUDL Data/sqlite',
 'parquet_dir': '/Users/andreaquevedo/Desktop/PUDL Data/parquet',
 'datapkg_dir': '/Users/andreaquevedo/Desktop/PUDL Data/datapkg',
 'notebook_dir': '/Users/andreaquevedo/Desktop/PUDL Data/notebook',
 'ferc1_db': 'sqlite:////Users/andreaquevedo/Desktop/PUDL Data/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////Users/andreaquevedo/Desktop/PUDL Data/sqlite/pudl.sqlite'}

In [3]:
#Creating an engine instance for the sqlite pudl databse
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [4]:
#Checking the tables available in the database 
metadata = MetaData()
metadata.reflect(pudl_engine)
metadata.tables.keys()

dict_keys(['boiler_fuel_eia923', 'fuel_type_eia923', 'boiler_generator_assn_eia860', 'generators_entity_eia', 'plants_entity_eia', 'boilers_entity_eia', 'coalmine_eia923', 'energy_source_eia923', 'ferc_accounts', 'ferc_depreciation_lines', 'fuel_ferc1', 'plants_ferc1', 'utilities_ferc1', 'utilities_pudl', 'plants_pudl', 'fuel_receipts_costs_eia923', 'transport_modes_eia923', 'fuel_type_aer_eia923', 'generation_eia923', 'generation_fuel_eia923', 'prime_movers_eia923', 'generators_eia860', 'utilities_entity_eia', 'ownership_eia860', 'plant_in_service_ferc1', 'plants_eia', 'plants_eia860', 'plants_hydro_ferc1', 'plants_pumped_storage_ferc1', 'plants_small_ferc1', 'plants_steam_ferc1', 'purchased_power_ferc1', 'utilities_eia', 'utilities_eia860', 'utility_plant_assn'])

### Selecting data with SQL queries

In [5]:
#Plant monthly fuel cost data
fuel_costs = pd.read_sql(
    """SELECT * FROM fuel_receipts_costs_eia923""", pudl_engine)

fuel_costs.sample(10)

,id,plant_id_eia,report_date,contract_type_code,contract_expiration_date,energy_source_code,fuel_type_code_pudl,fuel_group_code,fuel_group_code_simple,mine_id_pudl,supplier_name,fuel_qty_units,heat_content_mmbtu_per_unit,sulfur_content_pct,ash_content_pct,mercury_content_ppm,fuel_cost_per_mmbtu,primary_transportation_mode_code,secondary_transportation_mode_code,natural_gas_transport_code,natural_gas_delivery_contract_type_code,moisture_content_pct,chlorine_content_ppm
426196,426196,8845,2018-09-01,C,2019-01-01,BIT,coal,coal,coal,645.0,p&n coal company,9507.0,14.546,5.09,42.6,0.92,NaN,TR,None,None,None,5.93,0.0
422841,422841,7212,2018-08-01,S,None,NG,gas,natural_gas,natural_gas,NaN,virginia power services energy,1344.0,1.039,0.00,0.0,0.00,6.472,PL,None,interruptible,interruptible,NaN,NaN
209271,209271,3944,2012-10-01,C,2016-12-01,BIT,coal,coal,coal,481.0,anker,38862.0,24.720,3.00,13.4,NaN,NaN,TR,None,None,None,NaN,NaN
195683,195683,3457,2012-06-01,S,None,NG,gas,natural_gas,natural_gas,NaN,kinder morgan,38.0,1.053,0.00,0.0,0.00,2.428,PL,None,interruptible,None,NaN,NaN
235404,235404,874,2013-07-01,C,2013-09-01,SUB,coal,coal,coal,26.0,peabody coal sales,54111.0,17.310,0.20,4.5,NaN,NaN,RR,None,None,None,NaN,NaN
183328,183328,6137,2012-02-01,C,2012-12-01,BIT,coal,coal,coal,157.0,pattiki,34632.0,23.014,2.73,8.3,NaN,3.615,RR,None,None,None,NaN,NaN
150839,150839,7277,2011-07-01,S,None,NG,gas,natural_gas,natural_gas,NaN,hess corp energy marketing division,4311.0,1.013,0.00,0.0,NaN,4.857,PL,None,firm,None,NaN,NaN
57906,57906,6639,2009-12-01,S,None,PC,coal,petroleum_coke,petroleum_coke,NaN,koch carbon,15689.0,28.340,5.62,0.6,NaN,0.838,RV,None,None,None,NaN,NaN
201199,201199,60,2012-08-01,S,None,DFO,oil,petroleum,oil,NaN,hartland fuel products,926.0,5.754,0.00,0.0,0.00,24.332,TR,None,None,None,NaN,NaN
196711,196711,10416,2012-06-01,S,None,NG,gas,natural_gas,natural_gas,NaN,coral,273280.0,1.016,0.00,0.0,0.00,NaN,PL,None,firm,None,NaN,NaN


In [6]:
fuel_costs.shape

(436174, 23)

In [7]:
fuel_costs.plant_id_eia.nunique()

1690

In [8]:
#Plant monthly generation data
eia923_gen_fuel = pd.read_sql(
    """SELECT * FROM generation_fuel_eia923""", pudl_engine)

eia923_gen_fuel.sample(10)

,id,plant_id_eia,report_date,nuclear_unit_id,fuel_type,fuel_type_code_pudl,fuel_type_code_aer,prime_mover_code,fuel_consumed_units,fuel_consumed_for_electricity_units,fuel_mmbtu_per_unit,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,net_generation_mwh
677320,677321,6246,2014-05-01,NaN,NG,gas,NG,CA,0.0,0.0,0.000,0.0,0.0,23989.887
1354625,1354626,58183,2018-06-01,NaN,NG,gas,NG,ST,83568.0,3223.0,0.975,81479.0,3142.0,214.370
1041638,1041639,57908,2016-03-01,NaN,NG,gas,NG,CT,61882.0,27535.0,1.067,66028.0,29380.0,5429.090
1181937,1181938,56958,2017-10-01,NaN,LFG,gas,MLG,IC,47760.0,47760.0,0.512,24453.0,24453.0,2551.504
467847,467848,55944,2012-04-01,NaN,WND,wind,WND,WT,0.0,0.0,0.000,1795.0,1795.0,188.658
1029165,1029166,56804,2016-10-01,NaN,WND,wind,WND,WT,0.0,0.0,0.000,386913.0,386913.0,41910.000
1187711,1187712,57467,2017-12-01,NaN,WDS,waste,WWW,ST,11267.0,1864.0,10.344,116546.0,19281.0,3146.499
723905,723906,54800,2014-06-01,NaN,NG,gas,NG,GT,27758.0,10388.0,0.985,27342.0,10232.0,1817.448
301220,301221,10125,2011-09-01,NaN,DFO,oil,DFO,IC,11.0,6.0,5.880,65.0,36.0,5.785
491090,491091,58139,2012-03-01,NaN,NG,gas,NG,GT,42097.0,42097.0,1.035,43570.0,43570.0,3346.823


In [9]:
eia923_gen_fuel.shape

(1395852, 14)

In [10]:
eia923_gen_fuel.plant_id_eia.nunique()

9922

In [11]:
#Plant metadata including lat and long
plants_entity_eia = pd.read_sql(
    """SELECT * FROM plants_entity_eia""", pudl_engine)

plants_entity_eia.sample(10)

,plant_id_eia,plant_name_eia,balancing_authority_code,balancing_authority_name,city,county,ferc_cogen_status,ferc_exempt_wholesale_generator,ferc_small_power_producer,grid_voltage_kv,grid_voltage_2_kv,grid_voltage_3_kv,iso_rto_code,latitude,longitude,nerc_region,primary_purpose_naics_id,sector_name,sector_id,state,street_address,zip_code,timezone
11792,62382,GMP Solar/Storage-Ferrisburgh,ISNE,ISO New England Inc.,Ferrisburgh,Addison,False,False,False,12.47,NaN,NaN,None,44.133411,-73.144064,NPCC,22.0,Electric Utility,1.0,VT,10 Greenbush Road,5456.0,America/New_York
690,1123,Anita,MISO,Midcontinent Independent Transmission System O...,Anita,Cass,False,False,False,35.00,NaN,NaN,MISO,NaN,NaN,MRO,22.0,Electric Utility,1.0,IA,828 Main Street,50020.0,US/Central
6411,56793,Elm Creek Wind LLC,MISO,Midcontinent Independent Transmission System O...,Trimont,None,False,False,False,34.50,NaN,NaN,MISO,43.754200,-94.949700,MRO,22.0,IPP Non-CHP,2.0,MN,231 220th Street,56176.0,America/Chicago
6293,56626,K&D Energy LLC,MISO,Midcontinent Independent Transmission System O...,Worthington,Jackson,False,None,True,69.00,NaN,NaN,MISO,43.618100,-95.417200,MRO,22.0,IPP Non-CHP,2.0,MN,32804 780th Street,56187.0,America/Chicago
4016,50700,Lilliwaup Falls Generating,BPAT,Bonneville Power Administration,Lilliwaup,Mason,False,False,None,NaN,7.2,NaN,None,47.470600,-123.115300,WECC,22.0,IPP Non-CHP,2.0,WA,None,98555.0,America/Los_Angeles
2308,6194,Tolk,SWPP,Southwest Power Pool,Muleshoe,Lamb,False,False,False,230.00,NaN,NaN,SPP,34.186494,-102.569990,SPP,22.0,Electric Utility,1.0,TX,9m E. of Muleshoe on Hwy 70,79371.0,America/Chicago
10333,60840,"Goya Foods, Inc- Secaucus Solar",PJM,"PJM Interconnection, LLC",Secaucus,Hudson,False,False,False,0.48,NaN,NaN,None,40.773120,-74.077397,RFC,22.0,IPP Non-CHP,2.0,NJ,100 Seaview Drive,7094.0,America/New_York
9864,60349,"Hill AFB LFG Facility, Bldg #737",PACE,PacifiCorp - East,Hill AFB,Davis,False,False,True,12.47,69.0,138.0,None,41.129020,-111.964147,WECC,928.0,Commercial Non-CHP,4.0,UT,6100 Foulis Rd,84056.0,America/Denver
7046,57445,Sun Farmer I,None,None,Fleetwood,Ashe,False,False,True,7.20,NaN,NaN,None,NaN,NaN,SERC,22.0,None,2.0,NC,240 Hidden Pastures Rd,29626.0,US/Eastern
4229,52057,Herkimer,NYIS,New York Independent System Operator,Herkimer,Herkimer,False,False,True,46.30,NaN,NaN,NYISO,43.033577,-74.983391,NPCC,22.0,IPP Non-CHP,2.0,NY,3142 St Rd 28 N,13350.0,America/New_York


In [12]:
plants_entity_eia.shape

(12375, 23)

In [13]:
plants_entity_eia.plant_id_eia.nunique()

12375

### Data Wrangling

**EIA Monthly Generation Data**

In [14]:
#Dropping unnecessary columns 
eia923_gen_fuel.drop(['id', 'prime_mover_code', 'fuel_consumed_units', 'fuel_consumed_for_electricity_units',
                     'fuel_mmbtu_per_unit', 'fuel_type', 'fuel_type_code_aer', 'nuclear_unit_id'],
                     axis=1, inplace= True)

In [15]:
#Checking column types 
eia923_gen_fuel.dtypes

plant_id_eia                             int64
report_date                             object
fuel_type_code_pudl                     object
fuel_consumed_mmbtu                    float64
fuel_consumed_for_electricity_mmbtu    float64
net_generation_mwh                     float64
dtype: object

In [16]:
#Converting report_date to datetime format and creating year and month column
eia923_gen_fuel['report_date']= pd.to_datetime(eia923_gen_fuel['report_date']) 
eia923_gen_fuel['year'] = pd.DatetimeIndex(eia923_gen_fuel['report_date']).year
eia923_gen_fuel['month'] = pd.DatetimeIndex(eia923_gen_fuel['report_date']).month
eia923_gen_fuel.dtypes

plant_id_eia                                    int64
report_date                            datetime64[ns]
fuel_type_code_pudl                            object
fuel_consumed_mmbtu                           float64
fuel_consumed_for_electricity_mmbtu           float64
net_generation_mwh                            float64
year                                            int64
month                                           int64
dtype: object

In [17]:
#Sorting dataframes by plant_id and report date 
eia923_gen_fuel= eia923_gen_fuel.sort_values(by=['plant_id_eia', 'report_date'])

Yearly generation by fuel

In [18]:
#Group by year,plant_id, and fuel_type_code_pudl and aggregating fuel consumption and generation
eia923_fuel_year= eia923_gen_fuel.groupby(['year', 'plant_id_eia', 'fuel_type_code_pudl'],
                        as_index=False).agg({'fuel_consumed_mmbtu': 'sum',
                                             'fuel_consumed_for_electricity_mmbtu': 'sum',
                                             'net_generation_mwh': 'sum'})

#Checking first 20 rows
eia923_fuel_year.head(20)

,year,plant_id_eia,fuel_type_code_pudl,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,net_generation_mwh
0,2009,2,hydro,2758750.0,2758750.0,282659.000
1,2009,3,coal,79317545.0,79317545.0,7947703.984
2,2009,3,gas,43164660.0,43164660.0,6139198.016
3,2009,3,oil,0.0,0.0,0.000
4,2009,4,hydro,8427915.0,8427915.0,863516.000
5,2009,7,coal,3264270.0,3264270.0,218328.650
6,2009,7,gas,588095.0,588095.0,42765.680
7,2009,7,oil,2734.0,2734.0,185.848
8,2009,7,waste,31166.0,31166.0,2049.822
9,2009,8,coal,52599048.0,52599048.0,5198072.952


In [19]:
#checking only one plant (as example)
eia923_fuel_year.loc[eia923_fuel_year['plant_id_eia']==7]

,year,plant_id_eia,fuel_type_code_pudl,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,net_generation_mwh
5,2009,7,coal,3264270.0,3264270.0,218328.650
6,2009,7,gas,588095.0,588095.0,42765.680
7,2009,7,oil,2734.0,2734.0,185.848
8,2009,7,waste,31166.0,31166.0,2049.822
7778,2010,7,coal,3618555.0,3618555.0,217945.182
7779,2010,7,gas,171615.0,171615.0,10524.610
7780,2010,7,oil,0.0,0.0,0.000
7781,2010,7,waste,20056.0,20056.0,1190.208
15666,2011,7,coal,3226696.0,3226696.0,202801.917
15667,2011,7,gas,386545.0,386545.0,26184.340


In [20]:
generation_mwh_fuel_year= pd.pivot_table(eia923_fuel_year, values = 'net_generation_mwh', 
                     index=['plant_id_eia', 'year'],
                     columns = ['fuel_type_code_pudl']).reset_index()

In [21]:
generation_mwh_fuel_year['primary_fuel'] = generation_mwh_fuel_year[generation_mwh_fuel_year.columns.difference(['plant_id_eia',
                                                                                                  'year'])].idxmax(axis=1)

In [22]:
col_list= list(generation_mwh_fuel_year)
col_list.remove('plant_id_eia')
col_list.remove('year')
col_list.remove('primary_fuel')
print(col_list)

['coal', 'gas', 'hydro', 'nuclear', 'oil', 'other', 'solar', 'waste', 'wind']


In [23]:
generation_mwh_fuel_year['total_net_generation_mwh'] = generation_mwh_fuel_year[col_list].sum(axis=1)

In [24]:
generation_mwh_fuel_year.rename(columns={'coal': "coal_generation_mwh", 'gas': "gas_generation_mwh",
                              'hydro': "hydro_generation_mwh", 'nuclear': "nuclear_generation_mwh",
                              'oil': "oil_generation_mwh", 'other': "other_generation_mwh", 
                              'solar': "solar_generation_mwh", 'waste': 'waste_generation_mwh',
                              'wind': "wind_generation_mwh"}, inplace= True)

In [25]:
generation_mwh_fuel_year.head()

fuel_type_code_pudl,plant_id_eia,year,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,primary_fuel,total_net_generation_mwh
0,2,2009,NaN,NaN,282659.0,NaN,NaN,NaN,NaN,NaN,NaN,hydro,282659.0
1,2,2010,NaN,NaN,100981.0,NaN,NaN,NaN,NaN,NaN,NaN,hydro,100981.0
2,2,2011,NaN,NaN,129035.0,NaN,NaN,NaN,NaN,NaN,NaN,hydro,129035.0
3,2,2012,NaN,NaN,159180.0,NaN,NaN,NaN,NaN,NaN,NaN,hydro,159180.0
4,2,2013,NaN,NaN,211342.0,NaN,NaN,NaN,NaN,NaN,NaN,hydro,211342.0


In [26]:
#checking only one plant (as example)
generation_mwh_fuel_year.loc[generation_mwh_fuel_year['plant_id_eia']==7]

fuel_type_code_pudl,plant_id_eia,year,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,primary_fuel,total_net_generation_mwh
30,7,2009,218328.650,42765.680,NaN,NaN,185.848,NaN,NaN,2049.822,NaN,coal,263330.0
31,7,2010,217945.182,10524.610,NaN,NaN,0.000,NaN,NaN,1190.208,NaN,coal,229660.0
32,7,2011,202801.917,26184.340,NaN,NaN,0.000,NaN,NaN,1213.743,NaN,coal,230200.0
33,7,2012,48070.290,173398.330,NaN,NaN,0.000,NaN,NaN,631.380,NaN,gas,222100.0
34,7,2013,82694.496,99309.504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gas,182004.0
35,7,2014,155388.239,60073.850,NaN,NaN,NaN,NaN,NaN,1287.911,NaN,coal,216750.0
36,7,2015,15644.693,199010.307,NaN,NaN,NaN,NaN,NaN,0.000,NaN,gas,214655.0
37,7,2016,0.000,190978.000,NaN,NaN,NaN,NaN,NaN,0.000,NaN,gas,190978.0
38,7,2017,0.000,193273.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gas,193273.0
39,7,2018,NaN,188877.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,gas,188877.0


Monthly generation by fuel

In [27]:
#Group by year,month, plant_id, and fuel_type_code_pudl and aggregating fuel consumption and generation
eia923_fuel_month= eia923_gen_fuel.groupby(['year', 'month', 'plant_id_eia', 'fuel_type_code_pudl'],
                        as_index=False).agg({'fuel_consumed_mmbtu': 'sum',
                                             'fuel_consumed_for_electricity_mmbtu': 'sum',
                                             'net_generation_mwh': 'sum'})

#Checking first 20 rows
eia923_fuel_month.head(20)

,year,month,plant_id_eia,fuel_type_code_pudl,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,net_generation_mwh
0,2009,1,2,hydro,282335.0,282335.0,28927.787
1,2009,1,3,coal,8642439.0,8642439.0,875865.983
2,2009,1,3,gas,2001050.0,2001050.0,286801.017
3,2009,1,3,oil,0.0,0.0,0.000
4,2009,1,4,hydro,1010219.0,1010219.0,103506.000
5,2009,1,7,coal,560127.0,560127.0,39975.564
6,2009,1,7,gas,15439.0,15439.0,1101.871
7,2009,1,7,oil,1654.0,1654.0,118.059
8,2009,1,7,waste,1100.0,1100.0,78.506
9,2009,1,8,coal,4949654.0,4949654.0,495311.361


In [28]:
#checking only one plant (as example)
eia923_fuel_month.loc[eia923_fuel_month['plant_id_eia']==7]

,year,month,plant_id_eia,fuel_type_code_pudl,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,net_generation_mwh
5,2009,1,7,coal,560127.0,560127.0,39975.564
6,2009,1,7,gas,15439.0,15439.0,1101.871
7,2009,1,7,oil,1654.0,1654.0,118.059
8,2009,1,7,waste,1100.0,1100.0,78.506
7778,2009,2,7,coal,328824.0,328824.0,20852.106
7779,2009,2,7,gas,62.0,62.0,3.919
7780,2009,2,7,oil,0.0,0.0,0.000
7781,2009,2,7,waste,520.0,520.0,32.975
15551,2009,3,7,coal,319993.0,319993.0,21380.267
15552,2009,3,7,gas,28134.0,28134.0,1879.733


In [29]:
generation_mwh_fuel_month= pd.pivot_table(eia923_fuel_month, values = 'net_generation_mwh', 
                     index=['plant_id_eia', 'year', 'month'],
                     columns = ['fuel_type_code_pudl']).reset_index()

In [30]:
generation_mwh_fuel_month['primary_fuel'] = generation_mwh_fuel_month[generation_mwh_fuel_month.columns.difference(['plant_id_eia',
                                                                                                                    'year',
                                                                                                                    'month'])].idxmax(axis=1)

In [31]:
col_list= list(generation_mwh_fuel_month)
col_list.remove('plant_id_eia')
col_list.remove('year')
col_list.remove('month')
col_list.remove('primary_fuel')
print(col_list)

['coal', 'gas', 'hydro', 'nuclear', 'oil', 'other', 'solar', 'waste', 'wind']


In [32]:
generation_mwh_fuel_month['total_net_generation_mwh'] = generation_mwh_fuel_month[col_list].sum(axis=1)

In [33]:
generation_mwh_fuel_month.rename(columns={'coal': "coal_generation_mwh", 'gas': "gas_generation_mwh",
                                  'hydro': "hydro_generation_mwh", 'nuclear': "nuclear_generation_mwh",
                                  'oil': "oil_generation_mwh", 'other': "other_generation_mwh", 
                                  'solar': "solar_generation_mwh", 'waste': 'waste_generation_mwh',
                                  'wind': "wind_generation_mwh"}, inplace= True)

In [34]:
generation_mwh_fuel_month.head()

fuel_type_code_pudl,plant_id_eia,year,month,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,primary_fuel,total_net_generation_mwh
0,2,2009,1,NaN,NaN,28927.787,NaN,NaN,NaN,NaN,NaN,NaN,hydro,28927.787
1,2,2009,2,NaN,NaN,12227.884,NaN,NaN,NaN,NaN,NaN,NaN,hydro,12227.884
2,2,2009,3,NaN,NaN,26298.379,NaN,NaN,NaN,NaN,NaN,NaN,hydro,26298.379
3,2,2009,4,NaN,NaN,22594.291,NaN,NaN,NaN,NaN,NaN,NaN,hydro,22594.291
4,2,2009,5,NaN,NaN,29305.852,NaN,NaN,NaN,NaN,NaN,NaN,hydro,29305.852


In [35]:
#checking only one plant (as example)
generation_mwh_fuel_month.loc[generation_mwh_fuel_month['plant_id_eia']==7]

fuel_type_code_pudl,plant_id_eia,year,month,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,primary_fuel,total_net_generation_mwh
360,7,2009,1,39975.564,1101.871,NaN,NaN,118.059,NaN,NaN,78.506,NaN,coal,41274.0
361,7,2009,2,20852.106,3.919,NaN,NaN,0.000,NaN,NaN,32.975,NaN,coal,20889.0
362,7,2009,3,21380.267,1879.733,NaN,NaN,0.000,NaN,NaN,0.000,NaN,coal,23260.0
363,7,2009,4,5891.763,14051.237,NaN,NaN,0.000,NaN,NaN,0.000,NaN,gas,19943.0
364,7,2009,5,14579.632,9550.742,NaN,NaN,0.000,NaN,NaN,65.626,NaN,coal,24196.0
365,7,2009,6,20790.000,25.645,NaN,NaN,0.000,NaN,NaN,353.355,NaN,coal,21169.0
366,7,2009,7,16577.517,6280.318,NaN,NaN,0.000,NaN,NaN,239.165,NaN,coal,23097.0
367,7,2009,8,22010.843,1228.249,NaN,NaN,0.000,NaN,NaN,340.908,NaN,coal,23580.0
368,7,2009,9,16678.048,1446.678,NaN,NaN,67.789,NaN,NaN,321.485,NaN,coal,18514.0
369,7,2009,10,10190.522,6248.847,NaN,NaN,0.000,NaN,NaN,351.631,NaN,coal,16791.0


**EIA Plant Metadata**

In [36]:
#Sorting dataframes by plant_id 
plants_entity_eia= plants_entity_eia.sort_values(by=['plant_id_eia'])

In [37]:
plants_entity_eia.head()

,plant_id_eia,plant_name_eia,balancing_authority_code,balancing_authority_name,city,county,ferc_cogen_status,ferc_exempt_wholesale_generator,ferc_small_power_producer,grid_voltage_kv,grid_voltage_2_kv,grid_voltage_3_kv,iso_rto_code,latitude,longitude,nerc_region,primary_purpose_naics_id,sector_name,sector_id,state,street_address,zip_code,timezone
0,2,Bankhead Dam,SOCO,"Southern Company Services, Inc. - Trans",Northport,Tuscaloosa,False,False,False,115.0,NaN,NaN,None,33.458665,-87.356823,SERC,22.0,Electric Utility,1.0,AL,19001 Lock 17 Road,35476.0,America/Chicago
1,3,Barry,SOCO,"Southern Company Services, Inc. - Trans",Bucks,Mobile,False,False,False,230.0,NaN,NaN,None,31.006900,-88.010300,SERC,22.0,Electric Utility,1.0,AL,North Highway 43,36512.0,America/Chicago
2,4,Walter Bouldin Dam,SOCO,"Southern Company Services, Inc. - Trans",Wetumpka,Elmore,False,False,False,115.0,NaN,NaN,None,32.583889,-86.283056,SERC,22.0,Electric Utility,1.0,AL,750 Bouldin Dam Road,36092.0,America/Chicago
3,5,Chickasaw,None,None,None,None,None,None,None,NaN,NaN,NaN,None,30.763300,-88.060600,None,NaN,None,NaN,AL,None,None,America/Chicago
4,7,Gadsden,SOCO,"Southern Company Services, Inc. - Trans",Gadsden,Etowah,False,False,False,115.0,NaN,NaN,None,34.012800,-85.970800,SERC,22.0,Electric Utility,1.0,AL,1000 Goodyear Ave,35903.0,America/Chicago


**Fuel Costs**

In [38]:
#Dropping unnecessary columns 
fuel_costs.drop(['id', 'contract_type_code', 'contract_expiration_date', 'energy_source_code', 'fuel_group_code',
                'fuel_group_code_simple', 'mine_id_pudl', 'supplier_name', 'heat_content_mmbtu_per_unit',
                'sulfur_content_pct', 'ash_content_pct', 'mercury_content_ppm', 'primary_transportation_mode_code',
                'secondary_transportation_mode_code', 'natural_gas_transport_code', 'natural_gas_delivery_contract_type_code',
                'moisture_content_pct', 'chlorine_content_ppm'],axis=1, inplace= True)

In [39]:
fuel_costs.dtypes

plant_id_eia             int64
report_date             object
fuel_type_code_pudl     object
fuel_qty_units         float64
fuel_cost_per_mmbtu    float64
dtype: object

In [40]:
#Converting report_date to datetime format and creating year and month column
fuel_costs['report_date']= pd.to_datetime(fuel_costs['report_date']) 
fuel_costs['year'] = pd.DatetimeIndex(fuel_costs['report_date']).year
fuel_costs['month'] = pd.DatetimeIndex(fuel_costs['report_date']).month
fuel_costs.dtypes

plant_id_eia                    int64
report_date            datetime64[ns]
fuel_type_code_pudl            object
fuel_qty_units                float64
fuel_cost_per_mmbtu           float64
year                            int64
month                           int64
dtype: object

In [41]:
#Sorting dataframes by plant_id and report date 
fuel_costs= fuel_costs.sort_values(by=['plant_id_eia', 'report_date'])

In [42]:
#Group by year, month, plant_id, and fuel_type_code_pudl and aggregating fuel consumption and generation

#lambda function to compute the weighted mean:
wm = lambda x: np.average(x, weights=fuel_costs.loc[x.index, 'fuel_qty_units'])

fuel_costs_month= fuel_costs.groupby(['year', 'month', 'plant_id_eia', 'fuel_type_code_pudl']).agg(fuel_qty_units = ('fuel_qty_units','sum'),
                                                                                                   fuel_cost_per_mmbtu_wm =('fuel_cost_per_mmbtu', wm)).reset_index()
                
#Checking first 20 rows
fuel_costs_month.head(20)

,year,month,plant_id_eia,fuel_type_code_pudl,fuel_qty_units,fuel_cost_per_mmbtu_wm
0,2009,1,3,coal,381438.0,4.498458
1,2009,1,3,gas,1931268.0,6.792357
2,2009,1,7,coal,21205.0,3.976000
3,2009,1,7,gas,18464.0,5.687645
4,2009,1,8,coal,170117.0,3.418795
5,2009,1,8,oil,362.0,11.823000
6,2009,1,9,gas,14477.0,4.963000
7,2009,1,10,coal,106862.0,3.633927
8,2009,1,10,gas,172368.0,7.337000
9,2009,1,26,coal,359828.0,3.504549


In [43]:
fuel_costs_mmbtu_month= pd.pivot_table(fuel_costs_month, values = 'fuel_cost_per_mmbtu_wm', 
                     index=['plant_id_eia', 'year', 'month'],
                     columns = ['fuel_type_code_pudl']).reset_index()

fuel_costs_mmbtu_month.drop(['nan'],axis=1, inplace= True)

In [44]:
fuel_costs_mmbtu_month.head()

fuel_type_code_pudl,plant_id_eia,year,month,coal,gas,oil
0,3,2009,1,4.498458,6.792357,NaN
1,3,2009,2,4.086086,5.223498,NaN
2,3,2009,3,3.703871,4.555295,NaN
3,3,2009,4,3.871816,3.889918,NaN
4,3,2009,5,3.666749,4.221598,NaN


In [45]:
#checking only one plant (as example)
fuel_costs_mmbtu_month.loc[fuel_costs_mmbtu_month['plant_id_eia']==7]

fuel_type_code_pudl,plant_id_eia,year,month,coal,gas,oil
118,7,2009,1,3.976,5.687645,NaN
119,7,2009,2,3.916,4.911816,NaN
120,7,2009,3,3.584,4.296674,NaN
121,7,2009,4,3.184,3.793964,NaN
122,7,2009,5,3.388,3.884701,NaN
123,7,2009,6,3.437,3.833626,NaN
124,7,2009,7,3.620,3.547099,NaN
125,7,2009,8,3.299,3.522815,NaN
126,7,2009,9,3.146,3.301889,NaN
127,7,2009,10,3.016,4.213620,NaN


In [46]:
fuel_costs_mmbtu_month.dtypes

fuel_type_code_pudl
plant_id_eia      int64
year              int64
month             int64
coal            float64
gas             float64
oil             float64
dtype: object

In [47]:
fuel_costs_mmbtu_month.rename(columns={'coal': "coal_cost_mmbtu_wm", 'gas': "gas_cost_mmbtu_wm",
                                       'oil': "oil_cost_mmbtu_wm"}, inplace= True)

In [48]:
fuel_costs_mmbtu_month.head()

fuel_type_code_pudl,plant_id_eia,year,month,coal_cost_mmbtu_wm,gas_cost_mmbtu_wm,oil_cost_mmbtu_wm
0,3,2009,1,4.498458,6.792357,NaN
1,3,2009,2,4.086086,5.223498,NaN
2,3,2009,3,3.703871,4.555295,NaN
3,3,2009,4,3.871816,3.889918,NaN
4,3,2009,5,3.666749,4.221598,NaN


### Merging Datasets

**generation_mwh_fuel_year and plants_entity_eia**

In [49]:
#outer join (keeping plant_id's present in either datasets)
plant_complete = pd.merge(generation_mwh_fuel_year, plants_entity_eia, on='plant_id_eia', how='outer')

In [50]:
plant_complete.year = plant_complete.year.astype('Int64')

In [51]:
plant_complete.shape

(73426, 35)

In [52]:
plant_complete.plant_id_eia.nunique()

12375

In [53]:
#inner join (keeping plant_id's shared by both datasets)
plant_complete_fuel= pd.merge(generation_mwh_fuel_year, plants_entity_eia, on='plant_id_eia')

In [54]:
plant_complete_fuel.shape

(70973, 35)

In [55]:
plant_complete_fuel.plant_id_eia.nunique()

9922

In [56]:
#keeping only variables of interest 
plant_geo_generation = plant_complete_fuel[['plant_id_eia', 'plant_name_eia', 'year', 'primary_fuel','total_net_generation_mwh',
                                            "coal_generation_mwh", "gas_generation_mwh","hydro_generation_mwh", "nuclear_generation_mwh",
                                            "oil_generation_mwh", "other_generation_mwh", "solar_generation_mwh", 'waste_generation_mwh', 
                                            "wind_generation_mwh",'latitude', 'longitude', 'state', 'city', 'county','zip_code', 
                                            'primary_purpose_naics_id']]

In [57]:
plant_geo_generation.head()

,plant_id_eia,plant_name_eia,year,primary_fuel,total_net_generation_mwh,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,latitude,longitude,state,city,county,zip_code,primary_purpose_naics_id
0,2,Bankhead Dam,2009,hydro,282659.0,NaN,NaN,282659.0,NaN,NaN,NaN,NaN,NaN,NaN,33.458665,-87.356823,AL,Northport,Tuscaloosa,35476.0,22.0
1,2,Bankhead Dam,2010,hydro,100981.0,NaN,NaN,100981.0,NaN,NaN,NaN,NaN,NaN,NaN,33.458665,-87.356823,AL,Northport,Tuscaloosa,35476.0,22.0
2,2,Bankhead Dam,2011,hydro,129035.0,NaN,NaN,129035.0,NaN,NaN,NaN,NaN,NaN,NaN,33.458665,-87.356823,AL,Northport,Tuscaloosa,35476.0,22.0
3,2,Bankhead Dam,2012,hydro,159180.0,NaN,NaN,159180.0,NaN,NaN,NaN,NaN,NaN,NaN,33.458665,-87.356823,AL,Northport,Tuscaloosa,35476.0,22.0
4,2,Bankhead Dam,2013,hydro,211342.0,NaN,NaN,211342.0,NaN,NaN,NaN,NaN,NaN,NaN,33.458665,-87.356823,AL,Northport,Tuscaloosa,35476.0,22.0


In [58]:
#checking only one plant (as example)
plant_geo_generation.loc[plant_geo_generation['plant_id_eia']==7]

,plant_id_eia,plant_name_eia,year,primary_fuel,total_net_generation_mwh,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,latitude,longitude,state,city,county,zip_code,primary_purpose_naics_id
30,7,Gadsden,2009,coal,263330.0,218328.650,42765.680,NaN,NaN,185.848,NaN,NaN,2049.822,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
31,7,Gadsden,2010,coal,229660.0,217945.182,10524.610,NaN,NaN,0.000,NaN,NaN,1190.208,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
32,7,Gadsden,2011,coal,230200.0,202801.917,26184.340,NaN,NaN,0.000,NaN,NaN,1213.743,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
33,7,Gadsden,2012,gas,222100.0,48070.290,173398.330,NaN,NaN,0.000,NaN,NaN,631.380,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
34,7,Gadsden,2013,gas,182004.0,82694.496,99309.504,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
35,7,Gadsden,2014,coal,216750.0,155388.239,60073.850,NaN,NaN,NaN,NaN,NaN,1287.911,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
36,7,Gadsden,2015,gas,214655.0,15644.693,199010.307,NaN,NaN,NaN,NaN,NaN,0.000,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
37,7,Gadsden,2016,gas,190978.0,0.000,190978.000,NaN,NaN,NaN,NaN,NaN,0.000,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
38,7,Gadsden,2017,gas,193273.0,0.000,193273.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0
39,7,Gadsden,2018,gas,188877.0,NaN,188877.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.0128,-85.9708,AL,Gadsden,Etowah,35903.0,22.0


In [59]:
plant_geo_generation.to_csv('../Output/CSVs/pudl_generation_metadata.csv', index=False)

**generation_mwh_fuel_month and fuel_costs_mmbtu_month**

In [60]:
#inner join (keeping plant_id's shared by both datasets)
fuel_cost_generation= pd.merge(fuel_costs_mmbtu_month, generation_mwh_fuel_month,  on=['plant_id_eia', 'year', 'month'])

In [61]:
fuel_cost_generation.shape

(72038, 17)

In [62]:
fuel_cost_generation.plant_id_eia.nunique()

899

In [63]:
fuel_cost_generation.head()

fuel_type_code_pudl,plant_id_eia,year,month,coal_cost_mmbtu_wm,gas_cost_mmbtu_wm,oil_cost_mmbtu_wm,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,primary_fuel,total_net_generation_mwh
0,3,2009,1,4.498458,6.792357,NaN,875865.983,286801.017,NaN,NaN,0.0,NaN,NaN,NaN,NaN,coal,1162667.0
1,3,2009,2,4.086086,5.223498,NaN,687562.108,327890.892,NaN,NaN,0.0,NaN,NaN,NaN,NaN,coal,1015453.0
2,3,2009,3,3.703871,4.555295,NaN,630413.284,688308.716,NaN,NaN,0.0,NaN,NaN,NaN,NaN,gas,1318722.0
3,3,2009,4,3.871816,3.889918,NaN,441720.491,641086.509,NaN,NaN,0.0,NaN,NaN,NaN,NaN,gas,1082807.0
4,3,2009,5,3.666749,4.221598,NaN,789099.665,579403.335,NaN,NaN,0.0,NaN,NaN,NaN,NaN,coal,1368503.0


In [64]:
#checking only one plant (as example)
fuel_cost_generation.loc[fuel_cost_generation['plant_id_eia']==7]

fuel_type_code_pudl,plant_id_eia,year,month,coal_cost_mmbtu_wm,gas_cost_mmbtu_wm,oil_cost_mmbtu_wm,coal_generation_mwh,gas_generation_mwh,hydro_generation_mwh,nuclear_generation_mwh,oil_generation_mwh,other_generation_mwh,solar_generation_mwh,waste_generation_mwh,wind_generation_mwh,primary_fuel,total_net_generation_mwh
118,7,2009,1,3.976,5.687645,NaN,39975.564,1101.871,NaN,NaN,118.059,NaN,NaN,78.506,NaN,coal,41274.0
119,7,2009,2,3.916,4.911816,NaN,20852.106,3.919,NaN,NaN,0.000,NaN,NaN,32.975,NaN,coal,20889.0
120,7,2009,3,3.584,4.296674,NaN,21380.267,1879.733,NaN,NaN,0.000,NaN,NaN,0.000,NaN,coal,23260.0
121,7,2009,4,3.184,3.793964,NaN,5891.763,14051.237,NaN,NaN,0.000,NaN,NaN,0.000,NaN,gas,19943.0
122,7,2009,5,3.388,3.884701,NaN,14579.632,9550.742,NaN,NaN,0.000,NaN,NaN,65.626,NaN,coal,24196.0
123,7,2009,6,3.437,3.833626,NaN,20790.000,25.645,NaN,NaN,0.000,NaN,NaN,353.355,NaN,coal,21169.0
124,7,2009,7,3.620,3.547099,NaN,16577.517,6280.318,NaN,NaN,0.000,NaN,NaN,239.165,NaN,coal,23097.0
125,7,2009,8,3.299,3.522815,NaN,22010.843,1228.249,NaN,NaN,0.000,NaN,NaN,340.908,NaN,coal,23580.0
126,7,2009,9,3.146,3.301889,NaN,16678.048,1446.678,NaN,NaN,67.789,NaN,NaN,321.485,NaN,coal,18514.0
127,7,2009,10,3.016,4.213620,NaN,10190.522,6248.847,NaN,NaN,0.000,NaN,NaN,351.631,NaN,coal,16791.0


In [65]:
fuel_cost_generation.to_csv('../Output/CSVs/pudl_cost_generation.csv', index=False)